<a href="https://colab.research.google.com/github/ort-eila/csiseminar/blob/main/project/ste3_vit_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [79]:
import os
rgb_images_path = "/content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/JPG_RGB_224_224/labels/"


debug_rgb_images_path = "/content/gdrive/MyDrive/Seminar/project/immune_310_project/output_folder"
output_json_file = os.path.join(debug_rgb_images_path,'metadata.json')   # Output metadata JSON file name


In [80]:
# ! pip install datasets transformers
# !pip install --upgrade datasets


In [81]:
import datasets

print("datasets version:", datasets.__version__)


datasets version: 2.14.4


In [82]:
import os
os.listdir(debug_rgb_images_path)

['metadata.json', 'Validation', 'Train', 'Test']

In [83]:
os.path.basename(debug_rgb_images_path)

'output_folder'

In [88]:
import os
import json

# Define the root folder
root_folder = debug_rgb_images_path  # Replace with your root folder containing subfolders
output_json_file = os.path.join(root_folder,'metadata.json')   # Output metadata JSON file name

metadata = []

# Iterate through subfolders (train, validation, test)
for split_folder in os.listdir(root_folder):
    split_folder_path = os.path.join(root_folder, split_folder)

    if os.path.isdir(split_folder_path):
        for label in os.listdir(split_folder_path):
            label_folder = os.path.join(split_folder_path, label)

            if os.path.isdir(label_folder):
                for image_file in os.listdir(label_folder):
                    if image_file.endswith('.png'):
                        image_path = os.path.join(label_folder, image_file)
                        metadata.append({"image_path": image_path, "label": label, "split": split_folder})

# Save metadata as JSON file
with open(output_json_file, 'w') as json_file:
    json.dump(metadata, json_file, indent=4)

print("Metadata JSON file created:", output_json_file)


Metadata JSON file created: /content/gdrive/MyDrive/Seminar/project/immune_310_project/output_folder/metadata.json


In [89]:
output_json_file

'/content/gdrive/MyDrive/Seminar/project/immune_310_project/output_folder/metadata.json'

In [90]:
from datasets import load_dataset

# Load the dataset
ds = load_dataset("json", data_files=output_json_file)
# ,split=['train'])

# Print some information about the dataset
print(ds)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'image_path', 'split'],
        num_rows: 14
    })
})


In [91]:
ds['train']

Dataset({
    features: ['label', 'image_path', 'split'],
    num_rows: 14
})